Importing necessary modules

In [44]:
import cv2
import numpy as np
import requests
import shutil
import mediapipe as mp

ImportError: cannot import name 'implement_array_function' from 'numpy.core._multiarray_umath' (/home/ros/.local/lib/python3.10/site-packages/numpy/core/_multiarray_umath.cpython-310-x86_64-linux-gnu.so)

Get model files for pose networks

In [5]:
def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [23]:
proto_file = "pose_deploy.prototxt"
weights_file = "pose_iter_584000.caffemodel"
weights_file_url = "http://posefs1.perception.cs.cmu.edu/Users/tsimon/Projects/coco/data/models/coco/pose_iter_440000.caffemodel"

Load neural network

## Via openpose

In [38]:
try:
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
except:
    #download weights file
    file = download_file(weights_file_url)
    net = cv2.dnn.readNetFromCaffe(proto_file, file)

if net.empty():
    raise ValueError("Failed to load the model. Check the file paths.")
    

In [39]:
#input_layer_names = net.getLayerNames()
#id = net.getLayerId(input_layer_names[0])
#input_blob_shape = net.

#print("Input blob shape:", input_blob_shape)

Video capture or read from file

In [40]:
capture = True

Visualization structure

In [41]:
BODY_PARTS = {0: "Nose", 1: "Neck", 2: "Right Shoulder", 3: "Right Elbow", 
              4: "Right Wrist", 5: "Left Shoulder", 6: "Left Elbow", 
              7: "Left Wrist", 8: "Right Hip", 9: "Right Knee", 
              10: "Right Ankle", 11: "Left Hip", 12: "Left Knee", 
              13: "Left Ankle", 14: "Right Eye", 15: "Left Eye", 
              16: "Right Ear", 17: "Left Ear"}
POSE_PAIRS = [[1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7], 
              [1, 8], [8, 9], [9, 10], [1, 11], [11, 12], [12, 13], 
              [0, 14], [14, 16], [0, 15], [15, 17]]

Video capture code

In [ ]:
if capture:
    cap = cv2.VideoCapture("test1.mkv")
    threshold = 0.1
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_width = frame.shape[1]
        frame_height = frame.shape[0]
        #print(f"Frame w{frame_width} h{frame_height}")
        
        # Prepare the frame for input
        try:
            inp_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (1280//10, 720//10), (0, 0, 0), swapRB=False, crop=False)
        except:
            print("Bad blob")
        #print(inp_blob)
        if frame is None or frame.size == 0:
            print("Empty frame detected!")
            continue
        try:
            net.setInput(inp_blob)
        except:
            print("Bad input")

        # Get the output
        try:
            output = net.forward()
        except:
            print("Exception occured")
        # Extract points
        #points = []
        #for i in range(len(BODY_PARTS)):
        #    heat_map = output[0, i, :, :]
        #    _, conf, _, point = cv2.minMaxLoc(heat_map)
        #    x = int((frame_width * point[0]) / output.shape[3])
        #    y = int((frame_height * point[1]) / output.shape[2])
        #    if conf > threshold:
        #        points.append((x, y))
        #        cv2.circle(frame, (x, y), 5, (0, 255, 0), thickness=-1)
        #    else:
        #        points.append(None)
                    # Draw lines for pose
        #for pair in POSE_PAIRS:
        #    part_from = pair[0]
        #    part_to = pair[1]

        #    if points[part_from] and points[part_to]:
        #        cv2.line(frame, points[part_from], points[part_to], (0, 255, 255), 2)
        #        cv2.circle(frame, points[part_from], 5, (0, 0, 255), thickness=-1)
        #        cv2.circle(frame, points[part_to], 5, (0, 0, 255), thickness=-1)


        # Show the result
        cv2.imshow("Pose Estimation", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[ERROR:0@1176.108] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.108] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.108] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@1176.155] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.155] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.155] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion faile

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


[ERROR:0@1176.330] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.330] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.330] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@1176.358] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.358] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.358] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion faile

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


[ERROR:0@1176.558] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.558] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.558] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@1176.581] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1176.581] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1176.581] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion faile

...

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


[ERROR:0@1188.885] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1188.886] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1188.886] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@1188.907] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1188.907] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1188.907] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion faile

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


[ERROR:0@1189.105] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1189.105] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1189.105] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion failed) !blobs.empty() || inputs.size() > 1 in function 'getMemoryShapes'

[ERROR:0@1189.132] global net_impl.cpp:1162 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv1_1): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=0
[ERROR:0@1189.132] global net_impl.cpp:1168 getLayerShapesRecursively     input[0] = [ 1 3 72 128 ]
[ERROR:0@1189.133] global net_impl.cpp:1178 getLayerShapesRecursively Exception message: OpenCV(4.10.0) /io/opencv/modules/dnn/src/layers/convolution_layer.cpp:368: error: (-215:Assertion faile

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


## Via Mediapipe

In [ ]:
mp_pose = mp.solutions.pose